### Documentation pour l'explication du scraping du site FlashScore dans les archives de la Ligue 1

Dans un premier temps nous allons voir les importations qu'on aura besoin

In [23]:
from bs4 import BeautifulSoup #ezaezae
import pandas as pd #ezaezae
import time #ezaezae
import re #eazezae
from selenium import webdriver #zaezaezae

Nous avons choisi quelles informations nous voulons scraper dans le site donc nous créons des tableaux pour ces données

In [20]:
domiciles=[]
exterieurs=[]
scoredoms=[]
scoreexts=[]
resultats=[]
saisons=[]

Maintenant nous allons faire une boucle afin de récuperer les données sur plusieurs années donc differentes url.

    Pour acceder a tous les matchs nous devons appuyer sur le bouton affciher plus ce qui correspond à la première partie de notre boucle.
    
    Notre boucle imbriqué nous permettera de récuperer ensuite tous les matchs durant la saisons.
    
    Ensuite nous insérerons les données dans les tablaux créer au dessus.

In [21]:
for x in range(4,9) :

    driver=webdriver.Chrome()
    y=x+1
    driver.implicitly_wait(1)
    url= 'https://www.flashscore.fr/football/france/ligue-1-201'+str(x)+'-201'+str(y)+'/resultats/'
    driver.get(url)
    
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    time.sleep(5)
    Accept_button = soup.find('button', attrs={'id' :"onetrust-accept-btn-handler"})

    string = '$("#'+Accept_button.get('id')+'").click();'
    driver.execute_script(string)
    
    more_matchs = driver.find_element_by_partial_link_text('Montrer plus de matchs')
    present = True
    while present == True :
        time.sleep(5)
        try :
            more_matchs.click()
            print("Succès du clique sur Montrer plus de matchs")
        except:
            present = False
            print("Affichage complet de la saison fini on passe au scraping")
        
    
    time.sleep(5)
    soup1 = BeautifulSoup(driver.page_source,'html.parser')
    liste_matchs=soup1.find_all('div',class_=re.compile("event__match event__match--static"))
    nb_match = len(liste_matchs)
    print("Nombres de match trouvées : {}.".format(nb_match))
   
    for i in liste_matchs :
        saison="201"+str(x)+"-201"+str(y)
        
        domicile=i.find('div',class_=re.compile("event__participant event__participant--home"))
        domicile=domicile.text
         
        exterieur=i.find('div',class_=re.compile("event__participant event__participant--away"))
        exterieur=exterieur.text
        
        score=i.find('div', class_="event__scores fontBold")
        scoredom=score.span.text
        scoreext=score.span.next_sibling.next_sibling.text
        
        scoredom=int(scoredom)
        scoreext=int(scoreext)
        
        if scoredom > scoreext :
            resultat="VD"
        if scoredom < scoreext :
            resultat="VE"
        if scoredom == scoreext :
            resultat="N"
        
        domiciles.append(domicile)
        exterieurs.append(exterieur)
        scoredoms.append(scoredom)
        scoreexts.append(scoreext)
        resultats.append(resultat)
        saisons.append(saison)
        
    driver.close()

Affichage complet de la saison fini on passe au scraping
Nombres de match trouvées : 104.
Affichage complet de la saison fini on passe au scraping
Nombres de match trouvées : 105.
Affichage complet de la saison fini on passe au scraping
Nombres de match trouvées : 103.
Affichage complet de la saison fini on passe au scraping
Nombres de match trouvées : 107.
Affichage complet de la saison fini on passe au scraping
Nombres de match trouvées : 106.


On créer ensuite un DataFrame puis on l'ajoute dans un fichier csv afin d'analyser les données

In [22]:
matchs=pd.DataFrame({
    'saison':saisons,
    'equipe_dom':domiciles,
    'equipe_ext':exterieurs,
    'score_dom':scoredoms,
    'score_ext':scoreexts,
    'resultat':resultats})

matchs.to_csv(r'resultats_matchs.csv',sep=";",index=None)